In [1]:
import os
import pandas as pd
import edgar_funcs as ef

headers = {"User-Agent": "russ@sunriseanalysis.com"}
ticker = "WSM"

In [2]:
q, k = ef.get_quarterly_annual_accn_series(ticker)

AttributeError: module 'edgar_funcs' has no attribute 'get_quarterly_annual_accn_series'

In [ ]:
def explore_facts_dict(data) -> pd.DataFrame:
    rows = []

    for account_key, account_values in data.items():
        for unit_key, unit_values in account_values.items():
            for item in unit_values:
                row = {"account_key": account_key, "unit_key": unit_key}
                row.update(item)
                rows.append(row)

    df = pd.DataFrame(rows)
    return df

NameError: name 'pd' is not defined

In [ ]:
def split_and_process_facts(ticker) -> pd.DataFrame:
    tenQs, tenKs = ef.get_quarterly_annual_accn_series(ticker)
    facts = ef.get_facts_for_ticker(ticker).T
    facts.index.name = "fact"
    facts = facts["units"]
    facts_dict = facts.to_dict()
    facts_df = ef.explore_facts_dict(facts_dict)
    annual_df = facts_df[facts_df["accn"].isin(tenKs)]
    quarterly_df = facts_df[facts_df["accn"].isin(tenQs)].dropna(subset=["frame"])
    annual_df = annual_df.drop(
        ["accn", "start", "fy", "fp", "frame", "filed", "form"], axis=1
    )
    quarterly_df = quarterly_df.drop(
        ["accn", "start", "fy", "fp", "frame", "filed", "form"], axis=1
    )
    annual_df["fact__unit"] = annual_df["account_key"] + "__" + annual_df["unit_key"]
    quarterly_df["fact__unit"] = (
        quarterly_df["account_key"] + "__" + quarterly_df["unit_key"]
    )
    annual_df = annual_df.drop(["account_key", "unit_key"], axis=1)
    quarterly_df = quarterly_df.drop(["account_key", "unit_key"], axis=1)
    annual_pivot = annual_df.pivot_table(
        index="end", columns="fact__unit", values="val", aggfunc="first"
    )
    quarterly_pivot = quarterly_df.pivot_table(
        index="end", columns="fact__unit", values="val", aggfunc="first"
    )
    annual_pivot.index = pd.to_datetime(annual_pivot.index)
    quarterly_pivot.index = pd.to_datetime(quarterly_pivot.index)
    annual_recent = annual_pivot[annual_pivot.index.isin(tenKs.index)]
    quarterly_recent = quarterly_pivot[quarterly_pivot.index.isin(tenQs.index)]
    annual_recent = annual_recent.dropna(axis=1, how="all")
    quarterly_recent = quarterly_recent.dropna(axis=1, how="all")
    for col in annual_recent.columns:
        if "_USD" in col or "_shares" in col and annual_recent[col].abs().max() > 1_000:
            annual_recent[col] = annual_recent[col] / 1_000_000
            annual_recent.rename(columns={col: col + "(millions)"}, inplace=True)
    for col in quarterly_recent.columns:
        if (
            "_USD" in col
            or "_shares" in col
            and quarterly_recent[col].abs().max() > 1_000
        ):
            quarterly_recent[col] = quarterly_recent[col] / 1_000_000
            quarterly_recent.rename(columns={col: col + "(millions)"}, inplace=True)
    quarterly_recent.name = "quarterly DataFrame"
    return quarterly_recent, annual_recent

In [ ]:
q, a = split_and_process_facts(ticker)

In [ ]:
q

fact__unit,AccountsPayableCurrent__USD(millions),AccruedIncomeTaxesCurrent__USD(millions),AccruedLiabilitiesCurrent__USD(millions),AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment__USD(millions),AccumulatedOtherComprehensiveIncomeLossForeignCurrencyTranslationAdjustmentNetOfTax__USD(millions),AccumulatedOtherComprehensiveIncomeLossNetOfTax__USD(millions),AdditionalPaidInCapital__USD(millions),AdjustmentsRelatedToTaxWithholdingForShareBasedCompensation__USD(millions),AdjustmentsToAdditionalPaidInCapitalSharebasedCompensationRequisiteServicePeriodRecognitionValue__USD(millions),AdvertisingExpense__USD(millions),...,UnrecognizedTaxBenefitsIncomeTaxPenaltiesAndInterestAccrued__USD(millions),UnrecognizedTaxBenefitsIncreasesResultingFromCurrentPeriodTaxPositions__USD(millions),UnrecognizedTaxBenefitsIncreasesResultingFromPriorPeriodTaxPositions__USD(millions),UnrecognizedTaxBenefitsInterestOnIncomeTaxesAccrued__USD(millions),UnrecognizedTaxBenefitsReductionsResultingFromLapseOfApplicableStatuteOfLimitations__USD(millions),UnrecognizedTaxBenefitsThatWouldImpactEffectiveTaxRate__USD(millions),UnrecognizedTaxBenefits__USD(millions),VariableLeaseCost__USD(millions),WeightedAverageNumberOfDilutedSharesOutstanding__shares(millions),WeightedAverageNumberOfSharesOutstandingBasic__shares(millions)
end,,,,,,,,,,,,,,,,,,,,,
2010-01-31,188.241,48.260,107.710,NaN,10.387,NaN,448.848,-3.621,24.989,264.963,...,NaN,1.029,0.655,NaN,1.553,10.594,15.866,NaN,107.373,105.763
2010-05-02,177.938,4.278,78.093,NaN,12.082,NaN,460.713,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109.639,107.129
2010-08-01,184.135,17.507,83.188,NaN,11.196,NaN,462.106,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110.224,107.668
2010-10-31,202.888,39.025,94.033,NaN,11.898,NaN,465.937,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108.908,106.152
2011-01-30,227.963,41.997,122.440,1127.493,12.990,NaN,466.885,17.918,26.630,293.623,...,NaN,0.821,0.000,4.062,2.807,7.812,11.619,NaN,109.522,106.956
2011-05-01,192.439,7.625,70.508,NaN,15.377,NaN,471.498,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.183,104.918
2011-07-31,196.843,13.190,78.488,NaN,15.451,NaN,478.024,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,106.766,104.467
2011-10-30,220.689,17.152,89.117,NaN,13.266,NaN,480.292,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.721,103.651
2012-01-29,218.329,22.435,111.774,1199.805,12.590,NaN,478.720,11.656,24.336,301.316,...,NaN,1.329,0.379,3.983,0.864,6.738,10.023,NaN,106.582,104.352


In [ ]:
facts = ef.get_facts_for_ticker(ticker).T
facts.index.name = "fact"
facts = facts["units"]
facts_dict = facts.to_dict()
facts_df = explore_facts_dict(facts_dict)
facts_df

NameError: name 'ef' is not defined